In [ ]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
# Set figure parameters
sc.set_figure_params(dpi=100, vector_friendly=True) 
# vector_friendly=True rasterizes large objects (such as dots in a scatterplot as pixels).
# More at https://scanpy.readthedocs.io/en/stable/generated/scanpy.set_figure_params.html

In [ ]:
## read data
data = sc.read("/home/shared/spatial-workshop-GCB-2025/visium_v1_ustekinumab.h5ad")

ℹ️ AnnData stores data as a hierarchical array store HDF5. A brief introduction on what is usually stored and how it can be accessed is here: [AnnData](https://jupyterhub.ims.bio/user/robin/lab/tree/epyc/robin/GCB2025/spatial-workshop-GCB-2025/slides/anndata-brief.pdf).


In [ ]:
## Normalize to sum to 10000 to make total RNA content comparable and then log-normalize data

data.layers["counts"] = data.X.copy() # store raw counts if needed later
sc.pp.normalize_total(data, target_sum=1e4)
sc.pp.log1p(data)

In [ ]:
## Select highly variable genes

sc.pp.highly_variable_genes(data, n_top_genes=2000, batch_key="Slide_ID")
# highly variable genes are computed for each batch (batch_key) and then put together by taking Union

In [ ]:
## Compute principal components

sc.tl.pca(data, mask_var="highly_variable") # Uses only highly variable genes to compute PCs

In [ ]:
## Produce batch effect corrected PC embeddings

# We'll use Harmony for batch effect removal. It removes batch effects in principal components.
# Corrected principal components are saved at .obsm["X_pca_harmony"]

sc.external.pp.harmony_integrate(data, key="Slide_ID", max_iter_harmony=50)
# This uses harmonypy package: https://github.com/slowkow/harmonypy. All arguments part of "harmonypy.run_harmony" can be passed.

In [ ]:
## Compute UMAP

sc.pp.neighbors(data, use_rep="X_pca_harmony") # calculates neighborhood graph of spots using corrected PCs. use_rep can be any key in .obsm
sc.tl.umap(data)

In [ ]:
## Plot UMAP and color by batch
sc.pl.umap(data, color="Slide_ID")

For evaluation of batch effect removal, several metrics are available in [scib](https://scib.readthedocs.io/en/latest/). Often requires true label. Alternatively, cluster labels can be used as true label and batch key can be used as batch label. In this case, assessment will be about whether cluster labels are mixture of batches and not purely of some batches.


<u>Optional</u>: Perform clustering with sc.tl.leiden() and evaluate batch correction.

In [ ]:
## Perform Leien clustering with resolution of 1

sc.tl.leiden(data, resolution=1, key_added="leiden_1") 
# leiden_1 with cluster numbers (as strings: "0", "1"...) will be available in .obs

In [ ]:
data.obs["leiden_1"].value_counts()

Other general clustering approaches (KMeans, agglomerative clustering and DBSCAN etc.) are availble from [scikit-learn](https://scikit-learn.org/stable/modules/clustering.html).

In [ ]:
## show clusters on UMAP
sc.pl.umap(data, color="leiden_1")

<u>Questions</u>
1. Find optimal resolution for clustering
2. Use spatial information for clustering

## 1. Selecting resolution for leiden clustering

There is no good/settled answer. An idea would be to use silhouette_score. 

$$\text{Silhouette score }= \frac{(b-a)}{max(a,b)},$$

where, $a$: mean intra-cluster distance, $b$: mean inter-cluster distance. Value 1 is the best, indicating, mean inter-cluster distance is very high - meaning clusters are highly distinct. -1 would be mean perfectly overlapping clusters.

In [ ]:
from sklearn.metrics import silhouette_score

In [ ]:
## choose a range of resolutions to search from

resolutions = np.arange(0.1, 2.1, 0.25)
# resolutions = np.arange(0.1, 2.1, 0.1) # add more resolutions
scores = {} # a dictionary to store scores for each resolution
print(f"searching over resolutions: {resolutions}")


from tqdm import tqdm # notify about progress
for res in tqdm(resolutions):
    sc.tl.leiden(data, resolution=res, key_added="leiden_temp")
    
    score = silhouette_score(data.obsm["X_pca"], data.obs["leiden_temp"])
    n_clust = len(data.obs["leiden_temp"].unique())

    data.obs.pop("leiden_temp") # remove added leiden_temp column
    scores[res] = {
        "silhouette": score,
        "n_clusters": n_clust
    }

In [ ]:
scores

In [ ]:
## search between 0.1 and 0.6 again since 0.1 has the maximum silhoutte but number of clusters are 2.
resolutions = np.arange(0.1, 0.6, 0.05)
# resolutions = np.arange(0.1, 2.1, 0.1) # add more resolutions
scores = {} # a dictionary to store scores for each resolution
print(f"searching over resolutions: {resolutions}")

for res in tqdm(resolutions):
    sc.tl.leiden(data, resolution=res, key_added="leiden_temp")
    
    score = silhouette_score(data.obsm["X_pca"], data.obs["leiden_temp"])
    n_clust = len(data.obs["leiden_temp"].unique())

    data.obs.pop("leiden_temp") # remove added leiden_temp column
    scores[res] = {
        "silhouette": score,
        "n_clusters": n_clust
    }

In [ ]:
scores

In [ ]:
## redo clustering with "best" resolution
resolution=0.35
sc.tl.leiden(data, resolution=resolution, key_added="cluster") 

In [ ]:
data.obs["cluster"].value_counts()

In [ ]:
sc.pl.umap(data, color="cluster")

In [ ]:
sc.pl.umap(data, color="cluster", legend_loc="on data")


In [ ]:
## Scale counts: (X-mean)/st-dev
data.layers["scaled"] = sc.pp.scale(data, copy=True, max_value=10).X

In [ ]:
## compute marker genes

sc.tl.rank_genes_groups(data, groupby="cluster", method="wilcoxon") # computes one vs all
sc.pl.rank_genes_groups_dotplot(data, groupby="cluster", layer="scaled",
                               vmax=2, vmin=-1, cmap="RdBu_r") # plots from scaled layer between -2 and 2.
# Scaling is better for visualization of genes at different experession levels

In [ ]:
## cluster 6 likely denotes glomerular and cluster 7 likley a mixture of glomerular and other compartments.
## Visualize these two clusters

sc.set_figure_params(dpi=200) #  for better resolution
for slide in data.obs["Slide_ID"].unique()[0:5]:
    sub = data[data.obs["Slide_ID"]==slide]
    ordered_clusters = sub.obs["cluster"].cat.categories 
    ## store colors to plot
    colors = []
    for cluster in ordered_clusters:
        if cluster not in ["6", "7"]:
            colors.append("none") # no color
        elif cluster=="6":
            colors.append("black")
        else:
            colors.append("yellow")

    sub.uns["cluster_colors"] = colors

    sc.pl.spatial(sub, library_id=slide, color=["cluster", None], title=["Cluster", slide])
sc.set_figure_params(dpi=100) # reset to 100

Cluster 6 visiually colocalizes in glomeruli. 7 is mixed. There is also a lot of noise.

In practice, however, deciding resolution for clustring depends a lot on regions of interest (ROIs). For example, in [https://www.nature.com/articles/s41467-024-52525-w](https://www.nature.com/articles/s41467-024-52525-w), we were interested in glomeruli and subdomains of glomeruli as this is glomerulonephritis. Therefore, we opted for a high resolution (1.2) and then merged closely related clusters. This approach usually works well!

In [ ]:
resolution = 1.2
sc.tl.leiden(data, resolution=resolution, key_added=f"leiden_{resolution}")
sc.pl.umap(data, color=f"leiden_{resolution}")
sc.pl.umap(data, color=f"leiden_{resolution}", legend_loc="on data")

print("Cluster markers")
sc.tl.rank_genes_groups(data, groupby=f"leiden_{resolution}", method="wilcoxon") # computes one vs all
sc.pl.rank_genes_groups_dotplot(data, groupby=f"leiden_{resolution}", layer="scaled",
                               vmin=-2, vmax=2, cmap="RdBu_r")

Clusters 13 and 10 show presence of glomerular markers and cluster 13 shows a mixture of glomerular and immune cell-related markers.

In [ ]:
## Visualize these two clusters

sc.set_figure_params(dpi=200) #  for better resolution
for slide in data.obs["Slide_ID"].unique()[0:5]:
    sub = data[data.obs["Slide_ID"]==slide]
    ordered_clusters = sub.obs[f"leiden_{resolution}"].cat.categories 
    ## store colors to plot
    colors = []
    for cluster in ordered_clusters:
        if cluster not in ["13", "10"]:
            colors.append("none") # no color
        elif cluster=="10":
            colors.append("black")
        else:
            colors.append("yellow")

    sub.uns[f"leiden_{resolution}_colors"] = colors

    sc.pl.spatial(sub, library_id=slide, color=[f"leiden_{resolution}", None], title=[f"leiden_{resolution}", slide])
sc.set_figure_params(dpi=100) # reset to 100

Annoatations from the manuscript are available at /home/shared/spatial-workshop-GCB-2025/visium_v1_compartments_gex.csv

<u>Optional<u/>

1. A dictionary of kindey cell type marker genes is available here: /home/shared/spatial-workshop-GCB-2025/kidney_markers.json.  
Based on these lists, annotate above clusters. Is it possible to annotate all cleanly?

In [ ]:
## save processed anndata object with clustering info

data.write("visium_v1_processed.h5ad")

## 2. Using spatial Information: 

**2.1 SpatialLeiden**

An alternative of Leiden clustering is SpatialLeiden: [Paper](https://genomebiology.biomedcentral.com/articles/10.1186/s13059-025-03489-7), [Code](https://github.com/HiDiHlabs/SpatialLeiden), [Usage](https://spatialleiden.readthedocs.io/stable/usage.html). It integrates several spatial information at various steps of leiden clustering. Does using it improve clustering and homogeneity of markers?

In [ ]:
import squidpy as sq
import spatialleiden as sl

ℹ️ SpatialLeiden has two important hyperparameters:  
1. resolution: resolution for latent space (of gene expression) and spatial layer (coordinates)
2. layer_ratio: ratio of weighting if the latent space and spatial layers.  A higher ratio will increase relevance of the spatial neighbors and lead to more spatially homogeneous clusters

Also, since spot-coordinates are used for clustering and each slide has its own frame of reference, the current implementation is not useful for multi-slide dataset.

In [ ]:
## Select one slide
sub = data[data.obs["Slide_ID"]=="V4_B"]

In [ ]:
sq.gr.spatial_neighbors(sub, coord_type="generic", n_neighs=10)

resolution, layer_ratio = 1, 1.8
sl.spatialleiden(sub, resolution=resolution, layer_ratio=layer_ratio, 
                 directed=(False, True), key_added=f"spatialleiden_{resolution}_{layer_ratio}")

<u>Tasks</u>

1. SpatialLeiden also provides a nice way to select "best" resolution and layer_ratio. However, this requires information on the number of clusters. Run SpatialLeiden with n_clusters being the number of cluster obtained with Leiden. Check [Usage]() for script.

2. How does SpatialLeiden clustering compare with Leiden? For this, redoing Leiden clustering for this subset of data (object "sub") is necessary. Does it lead to homogeneous distribution of marker expression? How is the spatial distribution different?

**2.2 Spatial domain identification developed specifically for spatial transcriptomics**

Several more domain-specific spatial domain identification algorithms exist that work with Visium data. Carry on to notebok number 3 (3-visium-domains-2.ipynb or 3-visium-domains-2_complete.ipynb)